# Vaex introduction in 11 minutes 

[Because vaex goes up to 11](https://en.wikipedia.org/wiki/Up_to_eleven)

## DataFrame
Central to Vaex is the DataFrame (similar, but more efficient than a Pandas DataFrame), and we often use the variable `df` to represent it. A DataFrame is an efficient representation for a large tabular dataset, and has:

 * A number of columns, say `x`, `y` and `z`, which are:
   * Backed by a Numpy array;
   * Wrapped by an expression system e.g. `df.x`, `df['x']` or `df.col.x` is an Expression;
   * Columns/expression can perform lazy computations, e.g. `df.x * np.sin(df.y)` does nothing, until the result is needed.
 * A set of virtual columns, columns that are backed by a (lazy) computation, e.g. `df['r'] = df.x/df.y` 
 * A set of selections, that can be used to explore the dataset, e.g. `df.select(df.x < 0)`
 * Filtered DataFrames, that does not copy the data, `df_negative = df[df.x < 0]`
 
Lets start with an example dataset, which is included in Vaex.

In [ ]:
import vaex
df = vaex.example()
df  # Since this is the last statement in a cell, it will print the DataFrame in a nice HTML format. 

### Columns
The above preview shows that this dataset contains $> 300,000$ rows, and columns named x ,y, z (positions), vx, vy, vz (velocities), E (energy), L (angular momentum), and an id (subgroup of samples). When we print out a columns, we can see that it is not a Numpy array, but an Expression.

In [ ]:
df.x  # df.col.x or df['x'] are equivalent, but df.x may be preferred because it is more tab completion friendly or programming friendly respectively

Most Numpy functions (ufuncs) can be performed on expressions, and will not result in a direct result, but in a new expression.

In [ ]:
import numpy as np
np.sqrt(df.x**2 + df.y**2 + df.z**2)

### Virtual columns
Sometimes it is convenient to store an expression as a column. We call this a virtual column since it does not take up any memory, and is computed on the fly when needed. A virtual column is treated just as a normal column.

In [ ]:
df['r'] = np.sqrt(df.x**2 + df.y**2 + df.z**2)
df[['x', 'y', 'z', 'r']]

### Selections and filtering
Vaex can be efficient when exploring subsets of the data, for instance to remove outliers or to inspect only a part of the data. Instead of making copies, Vaex internally keeps track which rows are selected.

In [ ]:
df.select(df.x < 0)
df.evaluate(df.x, selection=True)

Selections are useful when you frequently modify the portion of the data you want to visualize, or when you want to efficiently compute statistics on several portions of the data effectively.

Alternatively, you can also create filtered datasets. This is similar to using Pandas, except that Vaex does not copy the data.

In [ ]:
df_negative = df[df.x < 0]
df_negative[['x', 'y', 'z', 'r']]

## Statistics on N-d grids

A core feature of Vaex is the extremely efficient calculation of statistics on N-dimensional grids. The is rather useful for making visualisations of large datasets.

In [ ]:
df.count(), df.mean(df.x), df.mean(df.x, selection=True)

Similar to SQL's groupby, Vaex uses the binby concept, which tells Vaex that a statistic should be calculated on a regular grid (for performance reasons)

In [ ]:
counts_x = df.count(binby=df.x, limits=[-10, 10], shape=64)
counts_x

This results in a Numpy array with the number counts in 64 bins distributed between x = -10, and x = 10. We can quickly visualize this using Matplotlib.

In [ ]:
import matplotlib.pylab as plt
plt.plot(np.linspace(-10, 10, 64), counts_x)
plt.show()

We can do the same in 2D as well (this can be generalized to N-D actually!), and display it with Matplotlib.

In [ ]:
xycounts = df.count(binby=[df.x, df.y], limits=[[-10, 10], [-10, 20]], shape=(64, 128))
xycounts

In [ ]:
plt.imshow(xycounts.T, origin='lower', extent=[-10, 10, -10, 20])
plt.show()

In [ ]:
v = np.sqrt(df.vx**2 + df.vy**2 + df.vz**2)
xy_mean_v = df.mean(v, binby=[df.x, df.y], limits=[[-10, 10], [-10, 20]], shape=(64, 128))
xy_mean_v

In [ ]:
plt.imshow(xy_mean_v.T, origin='lower', extent=[-10, 10, -10, 20])
plt.show()

Other statistics can be computed, such as:

* [DataFrame.count](api.rst#vaex.dataframe.DataFrame.count)
* [DataFrame.mean](api.rst#vaex.dataframe.DataFrame.mean)
* [DataFrame.std](api.rst#vaex.dataframe.DataFrame.std)
* [DataFrame.var](api.rst#vaex.dataframe.DataFrame.var)
* [DataFrame.median_approx](api.rst#vaex.dataframe.DataFrame.median_approx)
* [DataFrame.percentile_approx](api.rst#vaex.dataframe.DataFrame.percentile_approx)
* [DataFrame.mode](api.rst#vaex.dataframe.DataFrame.mode)
* [DataFrame.min](api.rst#vaex.dataframe.DataFrame.min)
* [DataFrame.max](api.rst#vaex.dataframe.DataFrame.max)
* [DataFrame.minmax](api.rst#vaex.dataframe.DataFrame.minmax)
* [DataFrame.mutual_information](api.rst#vaex.dataframe.DataFrame.mutual_information)
* [DataFrame.correlation](api.rst#vaex.dataframe.DataFrame.correlation)

Or see the full list at the [API docs](api.rst#statistics).

## Getting your data in

Before continuing with this tutorial, you may want to read in your own data. Ultimately, a Vaex DataFrame just wraps a set of Numpy arrays. If you can access your data as a set of Numpy arrays, you can easily construct a DataFrame using [from_arrays](api.rst#vaex.from_arrays).

In [ ]:
import vaex
import numpy as np
x = np.arange(5)
y = x**2
df = vaex.from_arrays(x=x, y=y)
df

Other quick ways to get your data in are:
 
  * [from_arrow_table](api.rst#vaex.from_arrow_table): [Arrow](https://arrow.apache.org/) table support
  * [from_csv](api.rst#vaex.from_csv): Comma separated files
  * [from_ascii](api.rst#vaex.from_ascii): Space/tab separated files
  * [from_pandas](api.rst#vaex.from_pandas): Converts a pandas DataFrame
  * [from_astropy_table](api.rst#vaex.from_astropy_table): Converts an astropy table

Exporting, or converting a DataFrame to a different datastructure is also quite easy:
 
 * [DataFrame.to_arrow_table](api.rst#vaex.dataframe.DataFrame.to_arrow_table)
 * [DataFrame.to_dask_array](api.rst#vaex.dataframe.DataFrame.to_dask_array) 
 * [DataFrame.to_pandas_df](api.rst#vaex.dataframe.DataFrame.to_pandas_df)
 * [DataFrame.export](api.rst#vaex.dataframe.DataFrameLocal.export)
 * [DataFrame.export_hdf5](api.rst#vaex.dataframe.DataFrameLocal.export_hdf5)
 * [DataFrame.export_arrow](api.rst#vaex.dataframe.DataFrameLocal.export_arrow)
 * [DataFrame.export_fits](api.rst#vaex.dataframe.DataFrameLocal.export_fits)
 

Nowadays, it is common to put data, especially larger dataset, on the cloud. Vaex can read data straight from S3, in a lazy manner, meaning that only that data that is needed will be downloaded, and cached on disk.

In [ ]:
# Read in the NYC Taxi dataset straight from S3
nyctaxi = vaex.open('s3://vaex/taxi/yellow_taxi_2009_2015_f32.hdf5?anon=true')
nyctaxi.head(5)

## Plotting

### 1-D and 2-D

Most visualizations are done in 1 or 2 dimensions, and Vaex nicely wraps Matplotlib to satisfy a variety of frequent use cases.


In [ ]:
import vaex
import numpy as np
df = vaex.example()

The simplest visualization is a 1-D plot using [DataFrame.plot1d](api.rst#vaex.dataframe.DataFrame.plot1d). When only given one argument, it will show a histogram showing 99.7% of the data.

In [ ]:
df.plot1d(df.x, limits='99.7%');

A slighly more complication visualization, is to plot not the counts, but a different statistic for that bin. In most 
cases, passing the `what='<statistic>(<expression>)` argument will do, where `<statistic>` is any of the statistics mentioned in the list above, or in the [API docs](api.rst#statistics).

In [ ]:
df.plot1d(df.x, what='mean(E)', limits='99.7%');

An equivalent method is to use the `vaex.stat.<statistic>` functions, e.g. [vaex.stat.mean](api.rst#vaex.stat.mean).

In [ ]:
df.plot1d(df.x, what=vaex.stat.mean(df.E), limits='99.7%');

The `vaex.stat.<statistic>` objects are very similar to Vaex expressions, in that they represent an underlying calculation. Typical arithmetic and Numpy functions can be applied to these calulations. However, these objects compute a single statistic, and do not return a column or expression.

In [ ]:
np.log(vaex.stat.mean(df.x)/vaex.stat.std(df.x))

These statistical objects can be passed to the `what` argument. The advantage being that the data will only have to be passed over once.

In [ ]:
df.plot1d(df.x, what=np.clip(np.log(-vaex.stat.mean(df.E)), 11, 11.4), limits='99.7%');

A similar result can be obtained by calculating the statistic ourselves, and passing it to plot1d's grid argument. Care has to be taken that the limits used for calculating the statistics and the plot are the same, otherwise the x axis may not correspond to the real data.

In [ ]:
limits = [-30, 30]
shape  = 64
meanE  = df.mean(df.E, binby=df.x, limits=limits, shape=shape)
grid   = np.clip(np.log(-meanE), 11, 11.4)
df.plot1d(df.x, grid=grid, limits=limits, ylabel='clipped E');

The same applies for 2-D plotting.

In [ ]:
df.plot(df.x, df.y, what=vaex.stat.mean(df.E)**2, limits='99.7%');

### Selections for plotting

While filtering is useful for narrowing down the contents of a DataFrame (e.g. `df_negative = df[df.x < 0]`) there are a few downsides to this. First, a practical issue is that when you filter 4 different ways, you will need to have 4 different DataFrames polluting your namespace. More importantly, when Vaex executes a bunch of statistical computations, it will do that per DataFrame, meaning that 4 passes over the data will be made, and even though all 4 of those DataFrames point to the same underlying data.

If instead we have 4 (named) selections in our DataFrame, we can calculate statistics in one single pass over the data, which can be significantly faster especially in the cases when your dataset is larger than your memory.

In the plot below we show three selection, which by default are blended together, requiring just one pass over the data.

In [ ]:
df.plot(df.x, df.y, what=np.log(vaex.stat.count()+1), limits='99.7%',
        selection=[None, df.x < df.y, df.x < -10]);

### Advanced Plotting
Lets say we would like to see two plots next to eachother. To achieve this we can pass a list of expression pairs.

In [ ]:
df.plot([["x", "y"], ["x", "z"]], limits='99.7%',
        title="Face on and edge on", figsize=(10,4));

By default, if you have multiple plots, they are shown as columns, multiple selections are overplotted, and multiple 'whats' (statistics) are shown as rows.

In [ ]:
df.plot([["x", "y"], ["x", "z"]], 
        limits='99.7%',
        what=[np.log(vaex.stat.count()+1), vaex.stat.mean(df.E)],
        selection=[None, df.x < df.y],
        title="Face on and edge on", figsize=(10,10));

Note that the selection has no effect in the bottom rows.

However, this behaviour can be changed using the `visual` argument.

In [ ]:
df.plot([["x", "y"], ["x", "z"]],
        limits='99.7%',
        what=vaex.stat.mean(df.E),
        selection=[None, df.Lz < 0],
        visual=dict(column='selection'),
        title="Face on and edge on", figsize=(10,10));

### Slices in a 3rd dimension
If a 3rd axis (z) is given, you can 'slice' through the data, displaying the z slices as rows. Note that here the rows are wrapped, which can be changed using the `wrap_columns` argument.

In [ ]:
df.plot("Lz", "E", 
        limits='99.7%',
        z="FeH:-2.5,-1,8", show=True, visual=dict(row="z"),
        figsize=(12,8), f="log", wrap_columns=3);

### Visualization of smaller datasets

Although Vaex focuses on large datasets, sometimes you end up with a fraction of the data (e.g. due to a selection) and you want to make a scatter plot. You can do so with the following approach:

In [ ]:
import vaex
df = vaex.example()

In [ ]:
import matplotlib.pylab as plt
x = df.evaluate("x", selection=df.Lz < -2500)
y = df.evaluate("y", selection=df.Lz < -2500)
plt.scatter(x, y, c="red", alpha=0.5, s=4);

In [ ]:
df.scatter(df.x, df.y, selection=df.Lz < -2500, c="red", alpha=0.5, s=4)
df.scatter(df.x, df.y, selection=df.Lz > 1500, c="green", alpha=0.5, s=4);

### In control

While Vaex provides a wrapper for Matplotlib, there are situations where you want to use the [DataFrame.plot](api.rst#vaex.dataframe.DataFrame.plot) method, but want to be in control of the plot. Vaex simply uses the current figure and axes objects, so that it is easy to do.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,7))
plt.sca(ax1)
selection = df.Lz < -2500
x = df[selection].x.evaluate()#selection=selection)
y = df[selection].y.evaluate()#selection=selection)
df.plot(df.x, df.y)
plt.scatter(x, y)
plt.xlabel('my own label $\gamma$')
plt.xlim(-20, 20)
plt.ylim(-20, 20)

plt.sca(ax2)
df.plot1d(df.x, label='counts', n=True)
x = np.linspace(-30, 30, 100)
std = df.std(df.x.expression)
y = np.exp(-(x**2/std**2/2)) / np.sqrt(2*np.pi) / std
plt.plot(x, y, label='gaussian fit')
plt.legend()
plt.show()

### Healpix (Plotting)
[Healpix](https://en.wikipedia.org/wiki/HEALPix) plotting is supported via the [healpy](https://healpy.readthedocs.io) package. Vaex does not need special support for healpix, only for plotting, but some helper functions are introduced to make working with healpix easier. 

In the following example we will use the TGAS astronomy dataset.
 
To understand healpix better, we will start from the beginning. If we want to make a density sky plot, we would like to pass healpy a 1D Numpy array where each value represents the density at a location of the sphere, where the location is determined by the array size (the healpix level) and the offset (the location). The TGAS (and Gaia) data includes the healpix index encoded in the `source_id`. By diving the `source_id` by 34359738368 you get a healpix index level 12, and diving it further will take you to lower levels.



In [ ]:
import vaex
import healpy as hp
tgas = vaex.datasets.tgas.fetch()

We will start showing how you could manually do statistics on healpix bins using `vaex.count`. We will do a really course healpix scheme (level 2).

In [ ]:
level = 2
factor = 34359738368 * (4**(12-level))
nmax = hp.nside2npix(2**level)
epsilon = 1e-16
counts = tgas.count(binby=tgas.source_id/factor, limits=[-epsilon, nmax-epsilon], shape=nmax)
counts

And using healpy's [mollview](https://healpy.readthedocs.io/en/latest/generated/healpy.visufunc.mollview.html#healpy.visufunc.mollview) we can visualize this.

In [ ]:
hp.mollview(counts, nest=True)

To simplify life, Vaex includes [DataFrame.healpix_count](api.rst#vaex.dataframe.DataFrame.healpix_count) to take care of this.

In [ ]:
counts = tgas.healpix_count(healpix_level=6)
hp.mollview(counts, nest=True)

Or even simpler, use [DataFrame.healpix_plot](api.rst#vaex.dataframe.DataFrame.healpix_plot)

In [ ]:
tgas.healpix_plot(f="log1p", healpix_level=6, figsize=(10,8),
                  healpix_output="ecliptic")

## [xarray](http://xarray.pydata.org) suppport

The `df.count` method can also return an [xarray](http://xarray.pydata.org) data array instead of a numpy array. This is easily done via the `array_type` keyword. Building on top of numpy, xarray adds dimension labels, coordinates and attributes, that makes working with multi-dimensional arrays more convenient. 

In [ ]:
xarr = df.count(binby=[df.x, df.y], limits=[-10, 10], shape=64, array_type='xarray')
xarr

In addition, xarray also has a plotting method that can be quite convenient. Since the xarray object has information about the labels of each dimension, the plot axis will be automatially labeled.

In [ ]:
xarr.plot();

Having xarray as output helps us to explore the contents of our data faster. In the following example we show how easy it is to plot the 2D distribution of the positions of the samples (x, y), per id group.

Notice how xarray automatically adds the appropriate titles and axis labels to the figure.

In [ ]:
df.categorize('id')  # treat the id as a categorical column - automatically adjusts limits and shape
xarr = df.count(binby=['x', 'y', 'id'], limits='95%', array_type='xarray')
np.log1p(xarr).plot(col='id', col_wrap=7);

## Interactive widgets

<div class="alert alert-info">

**Note:** The interactive widgets require a running Python kernel, if you are viewing this documentation online you can get a feeling for what the widgets can do, but computation will not be possible!

</div>

Using the `vaex-jupyter` package, we get access to interactive widgets.

In [ ]:
import vaex
import vaex.jupyter
import numpy as np
import pylab as plt
df = vaex.example()

The simplest way to get a more interactive visualization (or even print out statistics) is to use the `vaex.jupyter.interactive_selection` decorator, which will execute the decorated function each time the selection is changed.


In [ ]:
df.select(df.x > 0)
@vaex.jupyter.interactive_selection(df)
def plot(*args, **kwargs):
    print("Mean x for the selection is:", df.mean(df.x, selection=True))
    df.plot(df.x, df.y, what=np.log(vaex.stat.count()+1), selection=[None, True], limits='99.7%')
    plt.show()

After changing the selection programmatically, the visualization will update, as well as the print output.

In [ ]:
df.select(df.x > df.y)

However, to get truly interactive visualization, we need to use widgets, such as the [bqplot](https://github.com/bloomberg/bqplot) library. Again, if we make a selection here, the above visualization will also update, so lets select a square region.

One issue is that if you have installed ipywidget, bqplot, ipyvolume etc, it may not be enabled if you installed them from pip (installing from conda-forge will enable it automagically). To enable it, run the next cell, and refresh the notebook if they were not enabled already. *(Note that these commands will execute in the environment where the notebook is running, not where the kernel is running)*

In [ ]:
import sys
!jupyter nbextension enable --sys-prefix --py widgetsnbextension
!jupyter nbextension enable --sys-prefix --py bqplot
!jupyter nbextension enable --sys-prefix --py ipyvolume
!jupyter nbextension enable --sys-prefix --py ipympl
!jupyter nbextension enable --sys-prefix --py ipyleaflet


In [ ]:
# the default backend is bqplot, but we pass it here explicity
df.plot_widget(df.x, df.y, f='log1p', backend='bqplot')

## Joining

Joining in Vaex is similar to Pandas, except the data will no be copied. Internally an index array is kept for each row on the left DataFrame, pointing to the right DataFrame, requiring about 8GB for a billion row $10^9$ dataset.
Lets start with 2 small DataFrames, `df1` and `df2`:

In [ ]:
a = np.array(['a', 'b', 'c'])
x = np.arange(1,4)
df1 = vaex.from_arrays(a=a, x=x)
df1

In [ ]:
b = np.array(['a', 'b', 'd'])
y = x**2
df2 = vaex.from_arrays(b=b, y=y)
df2

The default join, is a 'left' join, where all rows for the left DataFrame (`df1`) are kept, and matching rows of the right DataFrame (`df2`) are added. We see that for the columns b and y, some values are missing, as expected.

In [ ]:
df1.join(df2, left_on='a', right_on='b')

A 'right' join, is basically the same, but now the roles of the left and right label swapped, so now we have some values from columns x and a missing.

In [ ]:
df1.join(df2, left_on='a', right_on='b', how='right')

We can also do 'inner' join, in which the output DataFrame has only the rows common between `df1` and `df2`.

In [ ]:
df1.join(df2, left_on='a', right_on='b', how='inner')

Other joins (e.g. outer) are currently not supported. Feel free to [open an issue on GitHub](https://github.com/vaexio/vaex/issues) for this.

## Group-by

With Vaex one can also do fast group-by aggregations. The output is Vaex DataFrame. Let us see few examples.

In [ ]:
import vaex
animal = ['dog', 'dog', 'cat', 'guinea pig', 'guinea pig', 'dog']
age = [2, 1, 5, 1, 3, 7]
cuteness = [9, 10, 5, 8, 4, 8]
df_pets = vaex.from_arrays(animal=animal, age=age, cuteness=cuteness)
df_pets

The syntax for doing group-by operations is virtually identical to that of Pandas. Note that when multiple aggregations are passed to a single column or expression, the output colums are appropriately named.

In [ ]:
df_pets.groupby(by='animal').agg({'age': 'mean',
                                  'cuteness': ['mean', 'std']})

Vaex supports a number of aggregation functions:

  * [vaex.agg.count](https://vaex.readthedocs.io/en/latest/api.html#vaex.agg.count): Number of elements in a group
  * [vaex.agg.first](https://vaex.readthedocs.io/en/latest/api.html#vaex.agg.first): The first element in a group
  * [vaex.agg.max](https://vaex.readthedocs.io/en/latest/api.html#vaex.agg.max): The largest value in a group
  * [vaex.agg.min](https://vaex.readthedocs.io/en/latest/api.html#vaex.agg.min): The smallest value in a group
  * [vaex.agg.sum](https://vaex.readthedocs.io/en/latest/api.html#vaex.agg.sum): The sum of a group
  * [vaex.agg.mean](https://vaex.readthedocs.io/en/latest/api.html#vaex.agg.mean): The mean value of a group
  * [vaex.agg.std](https://vaex.readthedocs.io/en/latest/api.html#vaex.agg.std): The standard deviation of a group
  * [vaex.agg.var](https://vaex.readthedocs.io/en/latest/api.html#vaex.agg.var): The variance of a group
  * [vaex.agg.nunique](https://vaex.readthedocs.io/en/latest/api.html#vaex.agg.nunique): Number of unique elements in a group

In addition, we can specify the aggregation operations inside the groupby-method. Also we can name the resulting aggregate columns as we wish. 

In [ ]:
df_pets.groupby(by='animal', 
                agg={'mean_age': vaex.agg.mean('age'),
                     'cuteness_unique_values': vaex.agg.nunique('cuteness'),
                     'cuteness_unique_min': vaex.agg.min('cuteness')})

A powerful feature of the aggregation functions in Vaex is that they support selections. This gives us the flexibility to make selections while aggregating. For example, let's calculate the mean cuteness of the pets in this example DataFrame, but separated by age.

In [ ]:
df_pets.groupby(by='animal', 
                agg={'mean_cuteness_old': vaex.agg.mean('cuteness', selection='age>=5'),
                     'mean_cuteness_young': vaex.agg.mean('cuteness', selection='~(age>=5)')})
                           

Note that in the last example, the grouped DataFrame contains NaNs for the groups in which there are no samples.

## String processing
String processing is similar to Pandas, except all operations are performed lazily, multithreaded, and faster (in C++). Check the [API docs](api.rst#string-operations) for more examples.

In [ ]:
import vaex
text = ['Something', 'very pretty', 'is coming', 'our', 'way.']
df = vaex.from_arrays(text=text)
df

In [ ]:
df.text.str.upper()

In [ ]:
df.text.str.title().str.replace('et', 'ET')

In [ ]:
df.text.str.contains('e')

In [ ]:
df.text.str.count('e')

## Propagation of uncertainties

In science one often deals with measurement uncertainties (sometimes refererred to as measurement errors). When transformations are made with quantities that have uncertainties associated with them, the uncertainties on these transformed quantities can be calculated automatically by Vaex. Note that propagation of uncertainties requires derivatives and matrix multiplications of lengthy equations, which is not complex, but tedious. Vaex can automatically calculate all dependencies, derivatives and compute the full covariance matrix.

As an example, let us use the TGAS astronomy dataset once again. Even though the TGAS dataset already contains galactic sky coordiantes (l and b), let's add them again by performing a coordinate system rotation from RA. and Dec.
We can apply a similar transformation and convert from the Sperical galactic to Cartesian coordinates.

In [ ]:
# convert parallas to distance
tgas.add_virtual_columns_distance_from_parallax(tgas.parallax)
# 'overwrite' the real columns 'l' and 'b' with virtual columns
tgas.add_virtual_columns_eq2gal('ra', 'dec', 'l', 'b')
# and combined with the galactic sky coordinates gives galactic cartesian coordinates of the stars
tgas.add_virtual_columns_spherical_to_cartesian(tgas.l, tgas.b, tgas.distance, 'x', 'y', 'z')

Since RA. and Dec. are in degrees, while ra_error and dec_error are in miliarcseconds, we need put them on the same scale

In [ ]:
tgas['ra_error'] = tgas.ra_error / 1000 / 3600
tgas['dec_error'] = tgas.dec_error / 1000 / 3600

We now let Vaex sort out what the covariance matrix is for the Cartesian coordinates x, y, and z. Then take 50 samples from the dataset for visualization.

In [ ]:
tgas.propagate_uncertainties([tgas.x, tgas.y, tgas.z])
tgas_50 = tgas.sample(50, random_state=42)

For this small subset of the dataset we can visualize the uncertainties, with and without the covariance.

In [ ]:
tgas_50.scatter(tgas_50.x, tgas_50.y, xerr=tgas_50.x_uncertainty, yerr=tgas_50.y_uncertainty)
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.show()
tgas_50.scatter(tgas_50.x, tgas_50.y, xerr=tgas_50.x_uncertainty, yerr=tgas_50.y_uncertainty, cov=tgas_50.y_x_covariance)
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.show()

From the second plot, we see that showing error ellipses (so narrow that they appear as lines) instead of error bars reveal that the distance information dominates the uncertainty in this case.

## Just-In-Time compilation

Let us start with a function that calculates the angular distance between two points on a surface of a sphere. The input of the function is a pair of 2 angular coordinates, in radians. 

In [ ]:
import vaex
import numpy as np
# From http://pythonhosted.org/pythran/MANUAL.html
def arc_distance(theta_1, phi_1, theta_2, phi_2):
    """
    Calculates the pairwise arc distance
    between all points in vector a and b.
    """
    temp = (np.sin((theta_2-2-theta_1)/2)**2
           + np.cos(theta_1)*np.cos(theta_2) * np.sin((phi_2-phi_1)/2)**2)
    distance_matrix = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return distance_matrix


Let us use the New York Taxi dataset of 2015, [as can be downloaded in hdf5 format](datasets.ipynb#New-york-taxi-dataset)

In [ ]:
nyctaxi = vaex.open('s3://vaex/taxi/yellow_taxi_2009_2015_f32.hdf5?anon=true')
# lets use just 20% of the data, since we want to make sure it fits
# into memory (so we don't measure just hdd/ssd speed)
nytaxi.set_active_fraction(0.2)

Although the function above expects Numpy arrays, Vaex can pass in columns or expression, which will delay the execution untill it is needed, and add the resulting expression as a virtual column.

In [ ]:
nytaxi['arc_distance'] = arc_distance(nytaxi.pickup_longitude * np.pi/180,
                                      nytaxi.pickup_latitude * np.pi/180,
                                      nytaxi.dropoff_longitude * np.pi/180,
                                      nytaxi.dropoff_latitude * np.pi/180)

When we calculate the mean angular distance of a taxi trip, we encounter some invalid data, that will give warnings, which we can safely ignore for this demonstration.

In [ ]:
%%time
nytaxi.mean(nytaxi.arc_distance)

This computation uses quite some heavy mathematical operations, and since it's (internally) using Numpy arrays, also uses quite some temporary arrays. We can optimize this calculation by doing a Just-In-Time compilation, based on [numba](https://numba.pydata.org/), [pythran](https://pythonhosted.org/pythran/), or if you happen to have an NVIDIA graphics card [cuda](https://developer.nvidia.com/cuda-zone). Choose whichever gives the best performance or is easiest to install.

In [ ]:
nytaxi['arc_distance_jit'] = nytaxi.arc_distance.jit_numba()
# nytaxi['arc_distance_jit'] = nytaxi.arc_distance.jit_pythran()
# nytaxi['arc_distance_jit'] = nytaxi.arc_distance.jit_cuda()

In [ ]:
%%time
nytaxi.mean(nytaxi.arc_distance_jit)

We can get a significant speedup ($\gt 4 x$) in this case.

## Parallel computations
As mentioned in the sections on selections, Vaex can do computations in parallel. Often this is taken care of, for instance, when passing multiple selections to a method, or multiple arguments to one of the statistical functions. However, sometimes it is difficult or impossible to express a computation in one expression, and we need to resort to doing so called 'delayed' computation, similar as in [joblib](https://pythonhosted.org/joblib) and [dask](https://dask.pydata.org).

In [ ]:
import vaex
df = vaex.example()
limits = [-10, 10]
delayed_count = df.count(df.E, binby=df.x, limits=limits,
                         shape=4, delay=True)
delayed_count

Note that now the returned value is now a promise (TODO: a more Pythonic way would be to return a Future). This may be subject to change, and the best way to work with this is to use the [delayed](api.rst#vaex.delayed) decorator. And call [DataFrame.execute](api.rst#vaex.dataframe.DataFrame.execute) when the result is needed.

In addition to the above delayed computation, we schedule more computation, such that both the count and mean are executed in parallel such that we only do a single pass over the data. We schedule the execution of two extra functions using the `vaex.delayed` decorator, and run the whole pipeline using `df.execute()`.

In [ ]:
delayed_sum = df.sum(df.E, binby=df.x, limits=limits,
                         shape=4, delay=True)

@vaex.delayed
def calculate_mean(sums, counts):
    print('calculating mean')
    return sums/counts

print('before calling mean')
# since calculate_mean is decorated with vaex.delayed
# this now also returns a 'delayed' object (a promise)
delayed_mean = calculate_mean(delayed_sum, delayed_count)

# if we'd like to perform operations on that, we can again
# use the same decorator
@vaex.delayed
def print_mean(means):
    print('means', means)
print_mean(delayed_mean)

print('before calling execute')
df.execute()

# Using the .get on the promise will also return the result
# However, this will only work after execute, and may be
# subject to change
means = delayed_mean.get()
print('same means', means)


## Extending Vaex
Vaex can be extended using several mechanisms.

### Adding functions
Use the [vaex.register_function decorator](api.rst#vaex.functions.register_function) API to add new functions. 

In [ ]:
import vaex
import numpy as np
@vaex.register_function()
def add_one(ar):
    return ar+1

The function can be invoked using the `df.func` accessor, to return a new expression. Each argument that is an expresssion, will be replaced by a Numpy array on evaluations in any Vaex context.

In [ ]:
df = vaex.from_arrays(x=np.arange(4))
df.func.add_one(df.x)

By default (passing `on_expression=True`), the function is also available as a method on Expressions, where the expression itself is automatically set as the first argument (since this is a quite common use case).

In [ ]:
df.x.add_one()

In case the first argument is not an expression, pass `on_expression=True`, and use `df.func.<funcname>`, to build a new expression using the function:

In [ ]:
@vaex.register_function(on_expression=False)
def addmul(a, b, x, y):
    return a*x + b * y

In [ ]:
df = vaex.from_arrays(x=np.arange(4))
df['y'] = df.x**2
df.func.addmul(2, 3, df.x, df.y)

These expressions can be added as virtual columns, as expected.

In [ ]:
df = vaex.from_arrays(x=np.arange(4))
df['y'] = df.x**2
df['z'] = df.func.addmul(2, 3, df.x, df.y)
df['w'] = df.x.add_one()
df

### Adding DataFrame accessors


When adding methods that operate on Dataframes, it makes sense to group them together in a single namespace.

In [ ]:
@vaex.register_dataframe_accessor('scale', override=True)
class ScalingOps(object):
    def __init__(self, df):
        self.df = df

    def mul(self, a):
        df = self.df.copy()
        for col in df.get_column_names(strings=False):
            if df[col].dtype:
                df[col] = df[col] * a
        return df

    def add(self, a):
        df = self.df.copy()
        for col in df.get_column_names(strings=False):
            if df[col].dtype:
                df[col] = df[col] + a
        return df
    

In [ ]:
df.scale.add(1)

In [ ]:
df.scale.mul(2)